In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_excel('AgencyDataset.xlsx', sheet_name='data')

# Display basic info
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           4600 non-null   datetime64[ns]
 1   price          4600 non-null   float64       
 2   bedrooms       4600 non-null   int64         
 3   bathrooms      4600 non-null   float64       
 4   sqft_living    4600 non-null   int64         
 5   sqft_lot       4600 non-null   int64         
 6   floors         4600 non-null   float64       
 7   waterfront     4600 non-null   int64         
 8   view           4600 non-null   int64         
 9   condition      4600 non-null   int64         
 10  sqft_above     4600 non-null   int64         
 11  sqft_basement  4600 non-null   int64         
 12  yr_built       4600 non-null   int64         
 13  yr_renovated   4600 non-null   int64         
 14  street         4600 non-null   object        
 15  city           4600 n

In [3]:
# Check for missing values
print(df.isnull().sum())

# For yr_renovated, replace 0 with NaN (since 0 likely means never renovated)
df['yr_renovated'] = df['yr_renovated'].replace(0, np.nan)

# For waterfront (binary), replace missing with 0 (assuming missing means no waterfront)
df['waterfront'] = df['waterfront'].fillna(0)

date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
street           0
city             0
statezip         0
country          0
dtype: int64


In [4]:
# Select numerical columns that might have outliers
num_cols = ['price', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement']

for col in num_cols:
    # Calculate IQR
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    # Define bounds (1.5*IQR is a common threshold)
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify outliers
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"\nNumber of outliers in {col}: {len(outliers)}")
    print(f"Lower bound: {lower_bound:.2f}, Upper bound: {upper_bound:.2f}")
    
    # Option 1: Cap outliers at the bounds
    df[col] = np.where(df[col] < lower_bound, lower_bound, 
                      np.where(df[col] > upper_bound, upper_bound, df[col]))
    
    # Option 2: Replace outliers with median (uncomment to use instead)
    # median_val = df[col].median()
    # df[col] = np.where((df[col] < lower_bound) | (df[col] > upper_bound), 
    #                   median_val, 
    #                   df[col])


Number of outliers in price: 240
Lower bound: -175256.25, Upper bound: 1153093.75

Number of outliers in sqft_living: 129
Lower bound: -280.00, Upper bound: 4360.00

Number of outliers in sqft_lot: 541
Lower bound: -4000.00, Upper bound: 20002.00

Number of outliers in sqft_above: 116
Lower bound: -475.00, Upper bound: 3965.00

Number of outliers in sqft_basement: 82
Lower bound: -915.00, Upper bound: 1525.00


In [5]:
# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

# Convert categorical columns to proper types
cat_cols = ['waterfront', 'view', 'condition']
df[cat_cols] = df[cat_cols].astype('category')

In [6]:
# Create age of property
df['age'] = df['date'].dt.year - df['yr_built']

# Create renovation flag
df['is_renovated'] = np.where(df['yr_renovated'].isna(), 0, 1)

# Create price per sqft
df['price_per_sqft'] = df['price'] / df['sqft_living']

# Create total rooms
df['total_rooms'] = df['bedrooms'] + df['bathrooms']

In [7]:
# Apply log transformation to skewed numerical features
skewed_cols = ['price', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement']
for col in skewed_cols:
    df[col+'_log'] = np.log1p(df[col])

In [8]:
# One-hot encode city (with top 10 most frequent cities)
top_cities = df['city'].value_counts().nlargest(10).index
for city in top_cities:
    df['city_'+city] = np.where(df['city']==city, 1, 0)

In [9]:
# Drop unnecessary columns
df = df.drop(['street', 'statezip', 'country'], axis=1)

# Save cleaned data
df.to_csv('cleaned_real_estate_data.csv', index=False)